In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('Bengaluru_House_Data.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
for column in data.columns:
    print(data[column].value_counts())
    print("*"*20)

In [ ]:
data.isna().sum() #for checking null values

In [ ]:
data.drop(columns=['society','balcony'],inplace=True)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data['location'].value_counts()

In [ ]:
data['location'] = data['location'].fillna('Sarjapur Road')

In [ ]:
data['area_type'].value_counts()

In [ ]:
data['size'] = data['size'].fillna('2 BHK')

In [ ]:
data['bath'] = data['bath'].fillna(data['bath'].median())

In [ ]:
data.info()

In [ ]:
data['bhk'] = data['size'].str.split().str.get(0).astype(int)
#In BHK column created on first column of size

In [ ]:
data[data.bhk > 20]

In [ ]:
data['total_sqft'].unique()

In [ ]:
def convertRange(x):
    temp = x.split('-')
    if len(temp) == 2:
        return (float(temp[0]) + float(temp[1]))/2   #mean
    try:
        return float(x)
    except:
        return None

In [ ]:
data['total_sqft']=data['total_sqft'].apply(convertRange)

In [ ]:
data.head()

Price Per Square Feet
#this column only for removing outliers

In [ ]:
data['price_per_sqft'] = data['price'] *100000/data['total_sqft']

In [ ]:
data['price_per_sqft']

In [ ]:
data.describe()

In [ ]:
data['location'].value_counts()

In [ ]:
data['location'] = data['location'].apply(lambda x:x.strip())  #strip for removing white-spaces
data['availability'] = data['availability'].apply(lambda x:x.strip())
data['area_type'] = data['area_type'].apply(lambda x:x.strip())
location_count=data['location'].value_counts()
location_count


In [ ]:
location_count_less_10=location_count[location_count<=10]
location_count_less_10

In [ ]:
data['location'] = data['location'].apply(lambda x: 'other'if x in location_count_less_10 else x)
data['location'].value_counts()
data['availability'].value_counts()

Outlier detection and removal

In [ ]:
data.describe()

In [ ]:
(data['total_sqft']/data['bhk']).describe()

In [ ]:
data=data[((data['total_sqft']/data['bhk']) >= 300)]
data.describe()

In [ ]:
data.shape

In [ ]:
data.price_per_sqft.describe()

In [ ]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        gen_df=subdf[(subdf.price_per_sqft > (m-st)) &(subdf.price_per_sqft <=(m+st))]
        df_output=pd.concat([df_output,gen_df],ignore_index=True)
    return df_output
data = remove_outliers_sqft(data)
data.describe()
#now here got data whose price near to mean minus standard deviation

In [ ]:
def bhk_outlier_remover(df):
    exclude_indices = np.array([])#for excluding
    for location,location_df in df.groupby('location'):
        bhk_states = {}
        for bhk,bhk_df in location_df.groupby('bhk'):
            bhk_states[bhk] = {
                'mean' : np.mean(bhk_df.price_per_sqft),
                'std' : np.std(bhk_df.price_per_sqft),
                'count' : bhk_df.shape[0]
            }
        for bhk,bhk_df in location_df.groupby('bhk'):
            stats=bhk_states.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft < (stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [ ]:
data=bhk_outlier_remover(data)

In [ ]:
data.shape

In [ ]:
data

In [ ]:
data.drop(columns=['size'],inplace=True)
data.drop(columns=['price_per_sqft'],inplace=True)


Cleaned Data

In [ ]:
data.head()

In [ ]:
data.to_csv("Cleaned_data.csv")


In [ ]:
X=data.drop(columns=['price'])
y=data['price']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge  #LinearRegression ,
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score# used to evaluate the performance of a regression model.

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0) 

In [ ]:
print(X_train.shape)
print(X_test.shape)

Applying Regrassion

In [ ]:
columns_trans = make_column_transformer(
    (OneHotEncoder(sparse=False,handle_unknown='ignore'), ['location']),
    (OneHotEncoder(sparse=False,handle_unknown='ignore'), ['availability']),
    (OneHotEncoder(sparse=False,handle_unknown='ignore'), ['area_type']),
   remainder='passthrough'  
)


In [ ]:
scaler= StandardScaler()

Applying Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso()

In [ ]:
pipe=make_pipeline(columns_trans,scaler,lasso)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred_lasso=pipe.predict(X_test)
r2_score(y_test,y_pred_lasso)

Applying Ridge

In [ ]:
ridge=Ridge()

In [ ]:
pipe=make_pipeline(columns_trans,scaler,ridge)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred_ridge = pipe.predict(X_test)
r2_score(y_test,y_pred_ridge)

In [ ]:
print("Lasso: ",r2_score(y_test,y_pred_lasso))
print("Ridge : ",r2_score(y_test,y_pred_ridge))

In [ ]:
import pickle 

In [ ]:
pickle.dump(pipe,open('RidgeModel.pkl','wb'))